In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import numpy as np
from transformers import BertModel, BertConfig
import pandas as pd
import matplotlib.pyplot as plt
import sys

# Loading data
In_set_file = loadmat('DLCB_dataset/O1_60/DLCB_input.mat')
Out_set_file = loadmat('DLCB_dataset/O1_60/DLCB_output.mat')

In_set = In_set_file['DL_input'].astype(np.float32)  # Convert to float32 if complex
Out_set = Out_set_file['DL_output'].astype(np.float32)  # Convert to float32 if complex

num_user_tot = In_set.shape[0]
DL_size_ratio = 0.2
DL_size = int(num_user_tot * DL_size_ratio)

np.random.seed(2016)
num_train = int(DL_size * 0.8)
num_test = int(num_user_tot * 0.2)

train_index, test_index = train_test_split(range(num_user_tot), test_size=num_test, random_state=2016)
In_train, In_test = In_set[train_index], In_set[test_index]
Out_train, Out_test = Out_set[train_index], Out_set[test_index]

# Number of base stations
num_base_stations = 4
training_history = [[] for _ in range(num_base_stations)]
# Divide input and output data into segments for each base station
input_segments_train = torch.chunk(torch.tensor(In_train), num_base_stations, dim=1)
output_segments_train = torch.chunk(torch.tensor(Out_train), num_base_stations, dim=1)




# Define the HybridBeamformingModel
class HybridBeamformingModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(HybridBeamformingModel, self).__init__()

        # Attention-based model
        self.attention_model = AttentionModel(input_size, output_size)

        # Feedforward model
        self.feedforward_model = FeedforwardModel(input_size, output_size)

        # Additional layers for combining outputs
        self.merge_fc1 = nn.Linear(output_size * 2, 128)
        self.merge_relu = nn.ReLU()
        self.merge_fc2 = nn.Linear(128, output_size)

    def forward(self, input_data):
        # Forward pass through attention-based model
        attention_output = self.attention_model(input_data)

        # Forward pass through feedforward model
        feedforward_output = self.feedforward_model(input_data)

        # Concatenate the outputs from both models
        combined_output = torch.cat([attention_output, feedforward_output], dim=-1)

        # Pass through additional layers for combining outputs
        merged_output = self.merge_fc1(combined_output)
        merged_output = self.merge_relu(merged_output)
        final_output = self.merge_fc2(merged_output)

        return final_output

# Attention-based model
class AttentionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(AttentionModel, self).__init__()

        # Attention mechanism
        self.attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=16)

        # Simple feedforward layer
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input_data):
        # Apply attention mechanism
        attention_output, _ = self.attention(input_data, input_data, input_data)

        # Take the output of the attention mechanism and pass it through a linear layer
        output = self.fc(attention_output)

        return output

# Feedforward model
class FeedforwardModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(FeedforwardModel, self).__init__()

        # Simplified feedforward layers
        self.fc1 = nn.Linear(input_size, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, output_size)

    def forward(self, input_data):
        output = self.fc1(input_data)
        output = self.relu(output)
        output = self.fc2(output)
        return output
print("Normal")
# List to store models for each base station
base_station_models = []

# Training each base station model
for base_station in range(num_base_stations):
    model = HybridBeamformingModel(input_size=64, output_size=512)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    train_dataset = TensorDataset(input_segments_train[base_station], output_segments_train[base_station])
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    num_epochs = 20
    total_batches = len(train_dataloader)
    
    for epoch in range(num_epochs):
        print(f"Training Base Station {base_station + 1}, Epoch {epoch + 1}")
        epoch_losses = []
        for batch_idx, (batch_inputs, batch_targets) in enumerate(train_dataloader):
            optimizer.zero_grad()
            predictions = model(batch_inputs)
            loss = criterion(predictions, batch_targets)
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())
            # Print progress
            percent_complete = (batch_idx + 1) / total_batches * 100
            sys.stdout.write(f"\rProgress: [{int(percent_complete)}%]")
            sys.stdout.flush()

        print()  # Move to the next line after completing an epoch
        training_history[base_station].append(np.mean(epoch_losses))

    base_station_models.append(model)

# Evaluation on the test set
input_segments_test = torch.chunk(torch.tensor(In_test), num_base_stations, dim=1)
output_segments_test = torch.chunk(torch.tensor(Out_test), num_base_stations, dim=1)

def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch_inputs, batch_targets in dataloader:
            predictions = model(batch_inputs)
            loss = criterion(predictions, batch_targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

base_station_losses = []
# Evaluate each base station model
for base_station, model in enumerate(base_station_models):
    test_dataset = TensorDataset(input_segments_test[base_station], output_segments_test[base_station])
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    test_loss = evaluate_model(model, test_dataloader)
    base_station_losses.append(test_loss)
    print(f"Base Station {base_station + 1} Test Loss: {test_loss}")

# Calculate the mean of MSE for all base stations
mean_mse = np.mean(base_station_losses)
print(f"Mean MSE across all Base Stations: {mean_mse}")

# Save the mean MSE to a CSV file
mean_mse_df = pd.DataFrame({"Mean_MSE": [mean_mse]})
mean_mse_df.to_csv("mean_mse_results.csv", index=False)

# Save the MSE values in the second sheet of mean_mse_results.csv
mean_mse_df = pd.read_csv("mean_mse_results.csv", index_col=None)

# Create a DataFrame for all training history
training_history_df = pd.DataFrame({"Epoch": range(1, num_epochs + 1)})

# Append training history to the DataFrame for each base station
for base_station, history in enumerate(training_history):
    sheet_name = f"Base_Station_{base_station + 1}_Training_History"
    training_history_df[sheet_name] = history

    # Optionally, you can append the sheet name to the DataFrame for reference
    mean_mse_df[sheet_name] = f"base_station_{base_station + 1}.csv"

# Save the updated DataFrame to mean_mse_results.csv
#mean_mse_df.to_csv("mean_mse_results.csv", index=False)

# Save the combined training history DataFrame to a single CSV file
training_history_df.to_csv("combined_training_history.csv", index=False)

# Aggregate predictions from all base station models
def predict(input_data):
    input_segments = torch.chunk(torch.tensor(input_data), num_base_stations, dim=1)
    predictions = []
    for base_station, model in enumerate(base_station_models):
        with torch.no_grad():
            model.eval()
            output = model(input_segments[base_station])
        predictions.append(output)
    return torch.cat(predictions, dim=1)

# Example usage
sample_input = np.random.rand(1, 256).astype(np.float32)  # Convert to float32 if not already
sample_input_tensor = torch.tensor(sample_input, dtype=torch.float32)  # Convert to torch tensor

predicted_output = predict(sample_input_tensor)

# Now 'predicted_output' contains the aggregated predictions from all base station models.
# You can further analyze or use these predictions as needed.
# Plotting the training history





plt.figure(figsize=(10, 6))
for base_station, history in enumerate(training_history):
    plt.plot(range(1, num_epochs + 1), history, label=f"Base Station {base_station + 1}")

plt.title('Training History of Beamforming Prediction Model')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True)

# Set both the lower and upper y-axis limits
plt.ylim(0.0005, plt.ylim()[1])

# Save the figure as a PDF
plt.savefig("training_history_plot.pdf", bbox_inches='tight')

# Show the plot
plt.show()



############ Attack the RF Beamforming Codeword Prediction Model ##########
print("Attack")
def fgsm_attack(model, input_data, target_data, epsilon):
    model.eval()
    input_data.requires_grad = True

    # Forward pass
    output = model(input_data)
    loss = nn.MSELoss()(output, target_data)

    # Backward pass
    model.zero_grad()
    loss.backward()

    # FGSM attack
    input_data_grad = input_data.grad.data
    perturbed_input = input_data + epsilon * torch.sign(input_data_grad)

    return perturbed_input

# Choose an epsilon value for the attack
epsilon = 0.05

# Generate adversarial examples for the test set
adversarial_examples = []
for base_station, model in enumerate(base_station_models):
    input_data = input_segments_test[base_station].clone().detach().requires_grad_(True)
    target_data = output_segments_test[base_station].clone().detach()

    perturbed_input = fgsm_attack(model, input_data, target_data, epsilon)
    adversarial_examples.append(perturbed_input)

# Evaluate the model on the adversarial examples
adversarial_losses = []
for base_station, model in enumerate(base_station_models):
    adversarial_dataset = TensorDataset(adversarial_examples[base_station], output_segments_test[base_station])
    adversarial_dataloader = DataLoader(adversarial_dataset, batch_size=32, shuffle=False)
    adversarial_loss = evaluate_model(model, adversarial_dataloader)
    adversarial_losses.append(adversarial_loss)
    print(f"Base Station {base_station + 1} Adversarial Test Loss: {adversarial_loss}")

# Calculate the mean of MSE for all base stations on adversarial examples
mean_adversarial_mse = np.mean(adversarial_losses)
print(f"Mean Adversarial MSE across all Base Stations: {mean_adversarial_mse}")

# Save the mean Adversarial MSE to a CSV file
mean_adversarial_mse_df = pd.DataFrame({"Mean_Adversarial_MSE": [mean_adversarial_mse]})
mean_adversarial_mse_df.to_csv("mean_adversarial_mse_results.csv", index=False)







############## Adversarial Training ######################
print("Adversarial Training")
# Define a new HybridBeamformingModel for adversarial training
adversarial_model = HybridBeamformingModel(input_size=64, output_size=512)
adversarial_optimizer = torch.optim.Adam(adversarial_model.parameters(), lr=1e-3)
adversarial_criterion = nn.MSELoss()


# Adversarial training loop with combined inputs
num_adversarial_epochs = 10
total_batches = len(train_dataloader)

for adversarial_epoch in range(num_adversarial_epochs):
    print(f"Adversarial Training Epoch {adversarial_epoch + 1}")
    adversarial_losses = []

    for batch_idx, (batch_inputs, batch_targets) in enumerate(train_dataloader):
        # Regular training step
        optimizer.zero_grad()
        predictions = model(batch_inputs)
        loss = criterion(predictions, batch_targets)
        loss.backward()
        optimizer.step()

        # Adversarial training step
        adversarial_inputs = fgsm_attack(adversarial_model, batch_inputs, batch_targets, epsilon)
        adversarial_predictions = adversarial_model(adversarial_inputs)
        adversarial_loss = adversarial_criterion(adversarial_predictions, batch_targets)
        adversarial_loss.backward()
        adversarial_optimizer.step()

        adversarial_losses.append(adversarial_loss.item())

        # Combine regular and adversarial inputs
        combined_inputs = torch.cat([batch_inputs, adversarial_inputs], dim=0)
        combined_targets = torch.cat([batch_targets, batch_targets], dim=0)

        # Training step with combined inputs
        optimizer.zero_grad()
        combined_predictions = model(combined_inputs)
        combined_loss = criterion(combined_predictions, combined_targets)
        combined_loss.backward()
        optimizer.step()

        # Print progress
        percent_complete = (batch_idx + 1) / total_batches * 100
        sys.stdout.write(f"\rProgress: [{int(percent_complete)}%]")
        sys.stdout.flush()

    print()  # Move to the next line after completing an epoch
    training_history[base_station].append(np.mean(adversarial_losses))


# Evaluate the adversarially trained model with combined inputs on the regular test set
adversarial_model.eval()

combined_test_losses = []
for base_station, model in enumerate(base_station_models):
    combined_test_dataset = TensorDataset(input_segments_test[base_station], output_segments_test[base_station])
    combined_test_dataloader = DataLoader(combined_test_dataset, batch_size=32, shuffle=False)
    combined_test_loss = evaluate_model(model, combined_test_dataloader)
    combined_test_losses.append(combined_test_loss)
    print(f"Base Station {base_station + 1} Combined Test Loss: {combined_test_loss}")

# Calculate the mean of MSE for all base stations on the regular test set after adversarial training with combined inputs
mean_combined_test_mse = np.mean(combined_test_losses)
print(f"Mean Combined Test MSE across all Base Stations: {mean_combined_test_mse}")

# Save the mean Combined Test MSE to a CSV file
mean_combined_test_mse_df = pd.DataFrame({"Mean_Combined_Test_MSE": [mean_combined_test_mse]})
mean_combined_test_mse_df.to_csv("mean_combined_test_mse_results.csv", index=False)

# Adversarial Training History Plot





Normal
Training Base Station 1, Epoch 1
Progress: [100%]
Training Base Station 1, Epoch 2
Progress: [44%]

/tmp/ipykernel_19958/1412494545.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  In_set = In_set_file['DL_input'].astype(np.float32)  # Convert to float32 if complex


Progress: [100%]
Training Base Station 1, Epoch 3
Progress: [100%]
Training Base Station 1, Epoch 4
Progress: [100%]
Training Base Station 1, Epoch 5
Progress: [100%]
Training Base Station 1, Epoch 6
Progress: [100%]
Training Base Station 1, Epoch 7
Progress: [100%]
Training Base Station 1, Epoch 8
Progress: [100%]
Training Base Station 1, Epoch 9
Progress: [100%]
Training Base Station 1, Epoch 10
Progress: [100%]
Training Base Station 1, Epoch 11
Progress: [100%]
Training Base Station 1, Epoch 12
Progress: [100%]
Training Base Station 1, Epoch 13
Progress: [100%]
Training Base Station 1, Epoch 14
Progress: [100%]
Training Base Station 1, Epoch 15
Progress: [100%]
Training Base Station 1, Epoch 16
Progress: [100%]
Training Base Station 1, Epoch 17
Progress: [100%]
Training Base Station 1, Epoch 18
Progress: [100%]
Training Base Station 1, Epoch 19
Progress: [100%]
Training Base Station 1, Epoch 20
Progress: [100%]
Training Base Station 2, Epoch 1
Progress: [100%]
Training Base Station 